Universidad Central de Venezuela <br>
Facultad de Ciencias <br>
Escuela de Computación <br>
Minería de Datos <br>
Profesor: Wilmer González <br>
Estudiante: Yarima Contreras. C.I.: V-29.706.291 <br>

# <center> Asignación 1: </center>
## <center> Pre-procesamiento de "Medical Students Dataset" </center>

Para iniciar la actividad asignada, se escogió utilizar la librería Polars, debido a sus características que proporcionan una mejor eficiencia en memoria. <br><br>Así, la instalación se realiza de la siguiente manera:

In [ ]:
!pip install polars

import polars as pl

Para la adquisición de los datos, se utilizó el dataset indicado, y se optó por usar la instrucción pl.scan_csv() para aprovechar las características de la librería Polars, en su modo Lazy, para una ejecución inteligente.
<br><br> De esta forma se realiza una primera inspección, visualizando el plan de ejecución, el esquema de la tabla a estudiar y una previsualización de los datos.

In [ ]:
PATH_DATA = r"D:\\Documentos\\Estudios\\UCV\\11. Semestre 2-2025\\Mineria de Datos\\DM Repo\\DM_UCV_YarimaContreras\\assignment1\\medical_students_dataset.csv"
lf = pl.scan_csv(PATH_DATA)

# Plan de ejecución
print(lf.explain())
print('\n')

# Esquema de la tabla
print(lf.head(0).collect().glimpse())
print('\n')

# Previsualización corta de los datos
with pl.Config(tbl_rows=10, tbl_cols=13):
    print(lf.collect())

Como primera inspección, se observan los tipos de datos descritos para cada columna. f64 son decimales de 64 bits, pero para campos como Student ID y Age, que almacenan números enteros, sería más eficiente definir las variables de tipo int. De igual manera, campos como Diabetes o Smoking, definidos como string, guardan valores de verdadero o falso, lo que hace que sea más conveniente manejarlos como bool/bit. Otro campo string es Gender, el cual está siendo indicado con los valores de "Male" y "Female", sin embargo, denotarlo con un char "M" y "F" es suficiente. Así, se realizan las siguientes acciones:

In [ ]:
# Conversión de las columnas Diabetes y Smoking a booleano
lf_bool = lf.with_columns([
    (pl.col("Diabetes") == "Yes").alias("Diabetes"),
    (pl.col("Smoking") == "Yes").alias("Smoking"),
])

# Cambio a char de Gender
lf_char = lf_bool.with_columns([
    pl.col("Gender").replace({
        "Female": "F",
        "Male": "M"
    }).alias("Gender")
])

# Casteo de Student ID y Age a enteros, y Gender a char. Diabetes y Smoking ya son booleanos 
lf_cast = lf_char.with_columns([
    pl.col("Student ID").cast(pl.Int32),
    pl.col("Age").cast(pl.Int8),
    pl.col("Gender").cast(pl.Categorical)
])

# Esquema de la tabla
print('\nEsquema actualizado de la tabla:')
print(lf_cast.head(0).collect().glimpse())

# Previsualización corta de los datos actualizados
with pl.Config(tbl_rows=10, tbl_cols=13):
    print(lf_cast.collect())

Teniendo esto, se procede a verificar la data contenida. La descripción del dataset indica la presencia de registros duplicados. Por esta razón, se ha decidido realizar un conteo de registros únicos, para eliminar dichos duplicados.

In [ ]:
print(f"Total de filas de la tabla: {lf.collect().height}")

lf_unique = lf_cast.unique()
print(f"Total de filas únicas de la tabla: {lf_unique.collect().height}")

Luego de garantizar que se trabaja con registros únicos, también se ha visualizado que las distintas columnas poseen cierta cantidad de valores vacíos. Por tanto, se ha escogido realizar el conteo de los mismos para decidir qué acción realizar:

In [ ]:
print("\nTotal de valores vacíos por columna:")
null_report = lf_unique.null_count().collect()
with pl.Config(tbl_rows=1, tbl_cols=13):
    print(null_report)

Gracias a esta consulta, se puede apreciar que la cantidad vacía de cada columna es aproximadamente del 10%. El que sea un valor consistente para todas las columnas pone en evidencia que es un dataset de data simulada, pero se ignora para este estudio.
<br><br>
Aunque una cifra cercana a 20000 registros es una cantidad considerable, dado que mediante la observación de las filas completas se encuentra que hay filas que unicamente poseen una columna con valor nulo, por tanto, no se considera viable eliminar todo el registro por esta razón. Sin embargo, sabiendo que la tabla posee 13 variables, si una fila posee 5 o más campos con valores vacíos, entonces probablemente no proporciona la información suficiente para contribuir a un modelo de Minería de Datos. Así, se ha tomado la decisión de descartar aquellas filas que posean 5 o más campos vacíos.

In [ ]:
lf_clean = lf_unique.filter(
    pl.all_horizontal(pl.sum_horizontal(pl.all().is_null()) <= 5)
)

# Verificación de resultados
total_before = lf_unique.select(pl.len()).collect().item()
total_after = lf_clean.select(pl.len()).collect().item()

print(f"Filas originales: {total_before}")
print(f"Filas tras la limpieza: {total_after}")
print(f"Filas eliminadas: {total_before - total_after}")

print("\nNuevo total de valores vacíos por columna:")
new_null_report = lf_clean.null_count().collect()
with pl.Config(tbl_rows=1, tbl_cols=13):
    print(new_null_report)

# Impresión de la tabla limpia
print("\nTabla limpia:")
with pl.Config(tbl_rows=199836, tbl_cols=13):
    print(lf_clean.collect())

Así, se han eliminado 164 registros que probablemente no poseían suficiencia de información.
<br><br>
Ahora, para las columnas restantes, sabiendo que el BMI (IMC - Índice de Masa Corporal) tiene como fórmula:
<center> IMC = peso (kg) / estatura² (m²) </center>
<br>
Entonces, si en las columnas de Height, Weight y BMI hace falta un único valor, es posible recuperarlo conociendo a los otros dos. Sin embargo, primero es necesario realizar un ajuste en el atributo Height, ya que se ha notado que el mismo se encuentra en centímetros (cm), pero para utilizar la fórmula es necesario que se encuentre en metros (m). Además, los tres campos poseen un exceso de decimales que es innecesario para el estudio.

In [ ]:
# Transformación de Height de centímetros a metros y redondeo inicial
lf_step1 = lf_unique.with_columns([
    (pl.col("Height") / 100).round(2).alias("Height"),
    pl.col("Weight").round(2),
    pl.col("BMI").round(2)
])

print("\nPaso 1:")
with pl.Config(tbl_rows=199836, tbl_cols=13):
    print(lf_step1.collect())

In [ ]:
# Calcular BMI nulos
lf_step2 = lf_step1.with_columns(
    pl.coalesce(
        pl.col("BMI"), 
        (pl.col("Weight") / (pl.col("Height")**2))
    ).round(2).alias("BMI")
)

print("\nPaso 2:")
with pl.Config(tbl_rows=199836, tbl_cols=13):
    print(lf_step2.collect())

In [ ]:
# Cálculo de Height y Weight faltantes
lf_step3 = lf_step2.with_columns([
    # Height = raíz(peso / BMI)
    pl.coalesce(
        pl.col("Height"), 
        (pl.col("Weight") / pl.col("BMI")).sqrt()
    ).round(2).alias("Height"),
    
    # Weight = BMI * altura^2
    pl.coalesce(
        pl.col("Weight"), 
        (pl.col("BMI") * (pl.col("Height")**2))
    ).round(2).alias("Weight")
])

print("\nPaso 3:")
with pl.Config(tbl_rows=199836, tbl_cols=13):
    print(lf_step3.collect())

In [ ]:
# Verificación final
print("Nulos después de cálculos de Height, Weight y BMI:")
print(lf_step3.select(["Height", "Weight", "BMI"]).null_count().collect())

Con estas transformaciones, se ha podido pasar de 19828, 19835 y 19812 valores nulos respectivamente, a 3761, 3771 y 3771. Estos últimos se han mantenido vacíos debido a que la información registrada no es suficiente para completar el cálculo. No obstante, se ha podido recuperar una cantidad considerable.

Así como fue realizado el redondeo para los campos de Height, Weith y BMI; Temperature también posee una gran cantidad de decimales que difícilmente sean significativos. En consecuencia, se toma la misma decisión de redondeo.

In [ ]:
lf_step4 = lf_step3.with_columns([
    pl.col("Temperature").round(2)
])

print("\nPaso 4:")
with pl.Config(tbl_rows=199836, tbl_cols=13):
    print(lf_step4.collect())

De esta manera se ha realizado el pre-procesamiento que se ha considerado necesario para el dataset asignado.

#### Fuentes Consultadas

* https://docs.pola.rs/user-guide/expressions/casting/
* https://docs.pola.rs/user-guide/expressions/categorical-data-and-enums/#category-ordering-and-comparison
* https://docs.pola.rs/api/python/dev/reference/expressions/api/polars.coalesce.html